<a href="https://colab.research.google.com/github/J-Gann/medfluencer/blob/main/medfluencer_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medfluencer Indexing

In [ ]:
!git clone https://github.com/J-Gann/medfluencer.git

Cloning into 'medfluencer'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 6), reused 20 (delta 2), pack-reused 0
Receiving objects: 100% (27/27), 924.21 KiB | 23.70 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Filtering content: 100% (5/5), 1003.46 MiB | 21.38 MiB/s, done.


In [ ]:
%pip install llama-index
%pip install -U llama-index-embeddings-huggingface
%pip install pinecone
%pip install -U llama-index-vector-stores-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 44.9 MB/s eta 0:00:00
  Using cached nvidia_cud

In [ ]:
import json
from llama_index.core.schema import TextNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from pinecone import Pinecone, ServerlessSpec
import pinecone
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.pinecone import PineconeVectorStore
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

ModuleNotFoundError: No module named 'llama_index.embeddings.huggingface'

In [ ]:
embed_model = HuggingFaceEmbedding(
    model_name="T-Systems-onsite/cross-en-de-roberta-sentence-transformer"
)

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

## Index Videos

In [ ]:
with open("medfluencer/scraping/videos_scraping.json") as f:
    video_data = json.load(f)

In [ ]:
splitter = SentenceSplitter(
    chunk_size=1024,    # smaller chunk size and larger overlap
    chunk_overlap=20,
)

video_nodes = []
for video_id, video in video_data.items():
    if video["transcription"] == None: continue
    node = TextNode(text=video["transcription"])
    video_nodes.append(node)

all_video_nodes = splitter.get_nodes_from_documents(video_nodes)

In [ ]:
pc.create_index(
    name="medfluencer-videos-index-t-systems",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [ ]:
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pc.Index("medfluencer-videos-index-t-systems"))
)

In [ ]:
index = VectorStoreIndex(all_video_nodes, storage_context=storage_context, embed_model=embed_model)

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/45 [00:00<?, ?it/s]

## Index Comments

In [ ]:
with open("medfluencer/scraping/comments_scraping.json") as f:
    comment_data = json.load(f)

In [ ]:
comment_nodes = []
for comment_id, comment in comment_data.items():
    if comment["text"] == None: continue
    if comment["text"] == "": continue
    node = TextNode(text=comment["text"])
    comment_nodes.append(node)

In [ ]:
pc.create_index(
    name="medfluencer-comments-index-t-systems",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [ ]:
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pc.Index("medfluencer-comments-index-t-systems"))
)

In [ ]:
index = VectorStoreIndex(comment_nodes, storage_context=storage_context, embed_model=embed_model)

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 03 Jul 2024 11:12:46 GMT', 'Content-Type': 'application/json', 'Content-Length': '182', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '228', 'x-pinecone-request-id': '741184817490599794', 'x-envoy-upstream-service-time': '4', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached the max storage allowed for this project (2 GB). To update or upsert new data, delete records or upgrade your plan.","details":[]}
